# Data Analysis with the OpenAI Assistants API

A tutorial on how to get GPT4 to analyse a CSV file with the new Assistants API.

OpenAI tells us "The Assistants API allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and knowledge to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, Retrieval, and Function calling."

In this tutorial we will use the Code Interpreter tool in order to read CSV data; we will then ask the AI questions about the data and display the messages that are returned.

Some of the code is adapted from the [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/Assistants_API_overview_python.ipynb) (MIT licence).

In [1]:
import json

# This utility function uses the IPython function 'display' to pretty-print the JSON of the
# various entities created in the Assistants API. This code is copied directly from the OpenAI Cookbook.

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

### Assistants
You can create Assistants directly through the Assistants API. You must provide a name, a model and instruction as in the code below and you must, of course be signed in. This Notebook requires you to enter your OpenAI API key.

After creating the assistant we display the JSON version of it.

In [2]:
from openai import OpenAI

# If you have set an environment variable set with your API key then you do not need
# to pass it to OpenAI
# client = OpenAI()

# Otherwise you need to pass the key as a parameter, e.g.
# api_key = "your key here - don't put this in publicly available code!"
# or enter it interactively
api_key = input("API key")

client = OpenAI(api_key = api_key)

In [3]:
# Create an assistant and show what it looks like

assistant = client.beta.assistants.create(
    name="Data Assistant",
    instructions="You are a data analyst who can analyse and interpret data files such as CSV and can draw conclusions from the data",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_Mqid9aVUOpaK3gLgmuHdFh7H',
 'created_at': 1705249345,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a data analyst who can analyse and interpret data files such as CSV and can draw conclusions from the data',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Data Assistant',
 'object': 'assistant',
 'tools': []}

### Create a thread

Initially, the thread is empty.


In [4]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_TVLDDcv8ZWAaKTbnAhBwjRJI',
 'created_at': 1705249350,
 'metadata': {},
 'object': 'thread'}

Then add the Message to the thread:


In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to find the mean value of a column in a pandas dataframe, can you help",
)
show_json(message)

{'id': 'msg_KIKyi9JoGPI88LqAAU0OS5ju',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to find the mean value of a column in a pandas dataframe, can you help'},
   'type': 'text'}],
 'created_at': 1705249356,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_TVLDDcv8ZWAaKTbnAhBwjRJI'}

In [6]:
show_json(thread)

{'id': 'thread_TVLDDcv8ZWAaKTbnAhBwjRJI',
 'created_at': 1705249350,
 'metadata': {},
 'object': 'thread'}

#### Runs


In [7]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_dQknLUHs0DuERN7QnmfO9nZk',
 'assistant_id': 'asst_Mqid9aVUOpaK3gLgmuHdFh7H',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1705250905,
 'expires_at': 1705251505,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a data analyst who can analyse and interpret data files such as CSV and can draw conclusions from the data',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_TVLDDcv8ZWAaKTbnAhBwjRJI',
 'tools': []}

A utility from the OpenAI Cookbook to poll the run for completion

In [8]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

Wait for completion of the run

In [9]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_dQknLUHs0DuERN7QnmfO9nZk',
 'assistant_id': 'asst_Mqid9aVUOpaK3gLgmuHdFh7H',
 'cancelled_at': None,
 'completed_at': 1705250916,
 'created_at': 1705250905,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a data analyst who can analyse and interpret data files such as CSV and can draw conclusions from the data',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1705250906,
 'status': 'completed',
 'thread_id': 'thread_TVLDDcv8ZWAaKTbnAhBwjRJI',
 'tools': []}

### Messages


In [10]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_OE189P7ud3lgNx6XaBym0Twh',
   'assistant_id': 'asst_Mqid9aVUOpaK3gLgmuHdFh7H',
   'content': [{'text': {'annotations': [],
      'value': 'Certainly! In order to calculate the mean value of a column in a pandas DataFrame, you can use the `.mean()` method. Here’s a step-by-step guide to doing just that:\n\nStep 1: First, ensure that you have pandas installed. If not, you can install it using `pip`:\n```bash\npip install pandas\n```\n\nStep 2: Import pandas in your Python script or notebook:\n```python\nimport pandas as pd\n```\n\nStep 3: Load your data into a pandas DataFrame. Assuming your data is in a CSV file, you can use the `read_csv` function:\n```python\ndf = pd.read_csv(\'path_to_your_csv_file.csv\')\n```\n\nStep 4: Calculate the mean of the desired column. Let’s say the column name is \'your_column_name\':\n```python\nmean_value = df[\'your_column_name\'].mean()\n```\n\nAnd that\'s it! The variable `mean_value` now holds the mean of the specified column.\n

In [11]:
# Pretty printing helper from the OpenAI Cookbook

def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

In [12]:
pretty_print(messages)

# Messages
assistant: Certainly! In order to calculate the mean value of a column in a pandas DataFrame, you can use the `.mean()` method. Here’s a step-by-step guide to doing just that:

Step 1: First, ensure that you have pandas installed. If not, you can install it using `pip`:
```bash
pip install pandas
```

Step 2: Import pandas in your Python script or notebook:
```python
import pandas as pd
```

Step 3: Load your data into a pandas DataFrame. Assuming your data is in a CSV file, you can use the `read_csv` function:
```python
df = pd.read_csv('path_to_your_csv_file.csv')
```

Step 4: Calculate the mean of the desired column. Let’s say the column name is 'your_column_name':
```python
mean_value = df['your_column_name'].mean()
```

And that's it! The variable `mean_value` now holds the mean of the specified column.

Here's an example with a complete snippet of code:

```python
import pandas as pd

# Load data
df = pd.read_csv('path_to_your_csv_file.csv')

# Calculate mean
mean_valu

Ask our Assistant a related question


In [ ]:
""" 
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user", 
    content="Are there other similar statistical methods for Pandas that will calculate mode, median, maximum and minimum?"
)

# Create the new run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)
"""

{'data': [{'id': 'msg_r2QmJiks0MVmIVXPxUoWXseu',
   'assistant_id': 'asst_y9cljnz0u79cbsCJWYF2dDBr',
   'content': [{'text': {'annotations': [],
      'value': 'Yes, pandas offers a range of statistical methods that can be used to calculate other descriptive statistics like mode, median, maximum, and minimum. Here are the methods and how you can use them:\n\n1. **Median** - To find the median value of a column:\n\n```python\nmedian_value = df[\'column_name\'].median()\nprint(f"The median value of the column is: {median_value}")\n```\n\n2. **Mode** - To find the mode value(s) of a column. This can return multiple values because a dataset can have more than one mode:\n\n```python\nmode_values = df[\'column_name\'].mode()\nprint("The mode value(s) of the column is/are:")\nprint(mode_values)\n```\n\n3. **Maximum** - To find the maximum value of a column:\n\n```python\nmax_value = df[\'column_name\'].max()\nprint(f"The maximum value of the column is: {max_value}")\n```\n\n4. **Minimum** - T

In [110]:
"""
pretty_print(messages)
"""

# Messages
assistant: Yes, pandas offers a range of statistical methods that can be used to calculate other descriptive statistics like mode, median, maximum, and minimum. Here are the methods and how you can use them:

1. **Median** - To find the median value of a column:

```python
median_value = df['column_name'].median()
print(f"The median value of the column is: {median_value}")
```

2. **Mode** - To find the mode value(s) of a column. This can return multiple values because a dataset can have more than one mode:

```python
mode_values = df['column_name'].mode()
print("The mode value(s) of the column is/are:")
print(mode_values)
```

3. **Maximum** - To find the maximum value of a column:

```python
max_value = df['column_name'].max()
print(f"The maximum value of the column is: {max_value}")
```

4. **Minimum** - To find the minimum value of a column:

```python
min_value = df['column_name'].min()
print(f"The minimum value of the column is: {min_value}")
```

Each of these methods

### Tools

A key feature of the Assistants API is the ability to equip our Assistants with Tools, like Code Interpreter, Retrieval, and custom Functions. Let's take a look at each.

#### Code Interpreter

Let's equip our Math Tutor with the [Code Interpreter](https://platform.openai.com/docs/assistants/tools/code-interpreter) tool, which we can do from the Dashboard...


In [13]:
assistant = client.beta.assistants.update(
    assistant.id,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)


{'id': 'asst_Mqid9aVUOpaK3gLgmuHdFh7H',
 'created_at': 1705249345,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a data analyst who can analyse and interpret data files such as CSV and can draw conclusions from the data',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Data Assistant',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}]}

In [14]:
# Upload the file
file = client.files.create(
    file=open(
        "europop.csv",
        "rb",
    ),
    purpose="assistants",
)
# Update Assistant
assistant = client.beta.assistants.update(
    assistant.id,
    tools=[{"type": "code_interpreter"}],
    file_ids=[file.id],
)
show_json(assistant)

{'id': 'asst_Mqid9aVUOpaK3gLgmuHdFh7H',
 'created_at': 1705249345,
 'description': None,
 'file_ids': ['file-2dDVMRoUs7Kj0s2rnhcthlun'],
 'instructions': 'You are a data analyst who can analyse and interpret data files such as CSV and can draw conclusions from the data',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Data Assistant',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}]}

In [15]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user", 
    content="""Please list the names, populations and percentage of the total, of the countries
               in order of descending population size?"""
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, 
    order="asc", 
    after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_RCiE6XPqbl6gHTZQyDyenjXz',
   'assistant_id': 'asst_Mqid9aVUOpaK3gLgmuHdFh7H',
   'content': [{'text': {'annotations': [],
      'value': "To achieve this, we will need to take the following steps:\n\n1. Load the data from the uploaded file into a pandas DataFrame.\n2. Identify the relevant columns for countries, populations, and calculate the percentages.\n3. Sort the DataFrame in descending order based on the population size.\n4. Calculate the percentage of the total for each country.\n5. Display the resulting DataFrame or list.\n\nLet's start by loading the data and looking at the first few rows to identify the columns we need. I'll proceed with loading the CSV file and displaying its head."},
     'type': 'text'}],
   'created_at': 1705254136,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_OaYkhNYmvK9RvUXOVsFpT2Gb',
   'thread_id': 'thread_TVLDDcv8ZWAaKTbnAhBwjRJI'},
  {'id': 'msg_3onCF0pdaygetzy8

In [16]:
show_json(messages)

{'data': [{'id': 'msg_RCiE6XPqbl6gHTZQyDyenjXz',
   'assistant_id': 'asst_Mqid9aVUOpaK3gLgmuHdFh7H',
   'content': [{'text': {'annotations': [],
      'value': "To achieve this, we will need to take the following steps:\n\n1. Load the data from the uploaded file into a pandas DataFrame.\n2. Identify the relevant columns for countries, populations, and calculate the percentages.\n3. Sort the DataFrame in descending order based on the population size.\n4. Calculate the percentage of the total for each country.\n5. Display the resulting DataFrame or list.\n\nLet's start by loading the data and looking at the first few rows to identify the columns we need. I'll proceed with loading the CSV file and displaying its head."},
     'type': 'text'}],
   'created_at': 1705254136,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_OaYkhNYmvK9RvUXOVsFpT2Gb',
   'thread_id': 'thread_TVLDDcv8ZWAaKTbnAhBwjRJI'},
  {'id': 'msg_3onCF0pdaygetzy8

In [17]:
pretty_print(messages)

# Messages
assistant: To achieve this, we will need to take the following steps:

1. Load the data from the uploaded file into a pandas DataFrame.
2. Identify the relevant columns for countries, populations, and calculate the percentages.
3. Sort the DataFrame in descending order based on the population size.
4. Calculate the percentage of the total for each country.
5. Display the resulting DataFrame or list.

Let's start by loading the data and looking at the first few rows to identify the columns we need. I'll proceed with loading the CSV file and displaying its head.
assistant: The file has been successfully loaded, and it appears to contain columns for the country names ('Country') and their respective populations ('Population (2021)'). 

Now, let's proceed with the following steps:
1. Sort the DataFrame based on 'Population (2021)' in descending order.
2. Calculate the percentage of the total population for each country.
3. List the names, populations, and percentages of the tota

In [116]:
# Create a message to append to our thread

#content = input("Enter query")
content = "What is the samllest country by population?"

In [117]:
message = client.beta.threads.messages.create(
    thread_id=thread.id, 
    role="user", 
    content=content
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, 
    order="asc", 
    after=message.id
)

pretty_print(messages)

# Messages
assistant: The smallest country by population, according to the provided list, is Malta, with a population of 526,751 in the year 2021.



In [118]:
# Clear up after session

deleted_assistant_file = client.beta.assistants.files.delete(
    assistant_id=assistant.id,
    file_id=file.id
)
print(deleted_assistant_file)

response = client.beta.assistants.delete(assistant.id)
print(response)



FileDeleteResponse(id='file-AzqzUD2oE3aPwcxUwA7Zw67h', deleted=True, object='assistant.file.deleted')
AssistantDeleted(id='asst_y9cljnz0u79cbsCJWYF2dDBr', deleted=True, object='assistant.deleted')
